# Basic Coral equation input example: Scalar, uncoupled PDE.

## Goals of the document

In this notebook, perhaps your first introduction to entering differential equations in Coral, we will learn the basics for equations implementation: 
+ defining parameters
+ distinguishing between linearly terms and quadratic contributions (and how to build the latter)
+ distinguishing between horizontally-averaged components and fluctuations around this mean

These concepts will be exemplified on a nonlinear diffusion equation for scalar quantities.

## Differential formulation

We consider the diffusion equation:
\begin{equation}
\frac{\partial}{\partial t} \phi= \boldsymbol{\nabla\cdot} \left[ D(\phi) \boldsymbol{\nabla} \phi \right]
\end{equation}
where $D(\phi)$ is the diffusivity coefficient. Since Coral can only handle quadratic nonlinearities, we choose the diffusivity to be of the form:
\begin{equation}
D(\phi) = D_0 + D_1 \phi
\end{equation}
with $D_0>0$ and $D_1\ll D_0$.

Anticipating that the horizontal and the vertical directions are inherently different from the quasi-inverse viewpoint, we introduce the horizontal laplacian 
\begin{equation}
\nabla^2_\perp = \partial_{xx} + \partial_{yy}
\end{equation}

* * *
## Quasi-Inverse formulation
As prescribed by the quasi-inverse method, we formally integrate with respect to $z$ the equation twice, in view of eliminating any z-derivative.
\begin{equation}
\frac{\partial}{\partial t}\iint \phi= D_0 \phi + \iint D_0 \nabla^2_\perp \phi  + \frac{D_1}{2} \iint \nabla_\perp^2 \phi^2 + \frac{D_1}{2} \phi^2 
\end{equation}

* * * 
# Formating the equations for Coral

## List of Parameters

Two physical parameters enter the diffusion coefficient: $D_0$ and $D_1$. We denote these parameters `D0` and `D1`. Additionally, all numerical prefactors have to be defined as named parameters. To illustrate this, we will define the parameter `oneHalf` In the equations`coral.equations`:
```
>>Param :: D0 := 10.d0 <<
>>Param :: D1 := 0.3d0 <<
>>Param :: oneHalf := 0.5d0 <<
```
Of course, numerical prefactors can often be absorbed in physical parameters. (An alternative to the declaration above would be to define a renormalized $D_1^* = D_1/2$.) However, this is not always possible, which is why we illustrate how to create a named parameter for numerical prefactors.

## List of variables
### Overview 
Coral require that the user defines four kind of variables:
+ `>>linear_variable_kxky` are fluctuating components of fields, that are linearly coupledIt is natural to think of these components in spectral space.
+ `>>linear_variable_mean` are horizontal-mean components of fields (i.e. functions of the vertical coordinate `z` only), that are linearly coupled. As with `>>linear_variable_mean`, those quantities are intrinsically spectral
+ `>>linear_variable_full` corresponds to fields that are computed in physical space. They are linear combinations of fluctuating and / or horizontal-mean components. (That is to say that they are necessarily formed by combining members from `>>linear_variable_kxky` and `>>linear_variable_mean`.) Their reason of being is to be building blocks for quadratic variables (also in physical space, see next bullet). Since they are computed in physical space, these `full` variables are available for output (in the forms of volumes, slices, profiles, or timeseries).
+ `>>quadratic_variable` are products of `>>linear_variable_full` and may represent advection. These `quadratic` variables are computed in physical space, and therefore are also available for output (in the forms of volumes, slices, profiles, or timeseries).

### Illustration on the diffusion equation example


We introduce the horizontal-average $\overline{\phi}(z)=\left\langle \phi\right\rangle_{x,y}$ and the fluctuations around this mean $\widetilde{\phi}(x,y,z)=\phi(x,y,z) - \overline{\phi}(z)$, which are treated as distinct linear variables. These variables, which we denote `phiMean` and `phi`, obey slightly different equations (obtained by projecting the diffusion equation above):
\begin{gather}
\frac{\partial}{\partial t}\iint \widetilde{\phi}= D_0 \widetilde{\phi} + \iint D_0 \nabla^2_\perp \widetilde{\phi}  + \frac{D_1}{2} \iint \nabla_\perp^2 \widetilde{\phi^2}+ \frac{D_1}{2} \widetilde{\phi^2} \nonumber \\
\frac{\partial}{\partial t}\iint \overline{\phi}= D_0 \overline{\phi}  + \frac{D_1}{2} \overline{\phi^2} \nonumber
\end{gather}
Of course, the full field $\phi$, denoted `phiFull`, is recovered by merely computing the sum `phiMean + phi`. Finally, the quadratic variable `phi2` represents the square of `phiFull`. Hence the declaration of variables:
```
>>====================================<<
>>linear_variable_kxky :: phi <<
>>linear_variable_mean :: phiMean <<
>>====================================<<
>>linear_variable_full       :: phiFull <<
>>linear_variable_full_build :: + phiMean <<
>>linear_variable_full_build :: + phi <<
>>====================================<<
>>quadratic_variable :: phi2 := phiFull.phiFull <<
>>====================================<<
```
## Set of equations
### The finite wavenumber system
We start with the finite-wavenumber governing equation 
\begin{equation}
\frac{\partial}{\partial t}\iint \widetilde{\phi}= D_0 \widetilde{\phi} + \iint D_0 \nabla^2_\perp \widetilde{\phi}  + \frac{D_1}{2} \iint \nabla_\perp^2 \widetilde{\phi^2}+ \frac{D_1}{2} \widetilde{\phi^2} \nonumber \end{equation}
This equation is second order in z (indicated below by `>>new_equation :: 2 <<`). Linearly, only $\mathtt{phi}=\widetilde{\phi}$ is involved. This variable need to be supplemented with two boundary conditions, for instance a no-flux top and bottom: $\partial_z \phi = 0$ at $z=0,1$ (encoded by integer `21` in Coral).

```
>>====================================<<
>>add_set_of_coupled_kxky_equations <<
>>linearly_coupled_var :: phi <<
>>add_BC_for_this_var :: 21 <<
>>new_equation :: 2 <<
>>add_d/dt_term  :: + Iz.Iz.phi <<
>>add_rhs_linear :: + D0.phi <<
>>add_rhs_linear :: + D0.Iz.Iz.dx.dx.phi <<
>>add_rhs_linear :: + D0.Iz.Iz.dy.dy.phi <<
>>add_rhs_NL     :: + D1.oneHalf.Iz.Iz.dx.dx.phi2 <<
>>add_rhs_NL     :: + D1.oneHalf.Iz.Iz.dy.dy.phi2 <<
>>add_rhs_NL     :: + D1.oneHalf.phi2 <<
>>====================================<<
```
### The finite wavenumber system
Next, we enter the horizontally-averaged equation:
\begin{equation}
\frac{\partial}{\partial t}\iint \overline{\phi}= D_0 \overline{\phi} + \frac{D_1}{2} \overline{\phi^2} \nonumber \end{equation}
This equation is also second order in z. Linearly, only $\mathtt{phiMean}=\overline{\phi}$ is involved, and for consistency this variable follows the same stress-free boundary condition.

```
>>====================================<<
>>add_set_of_coupled_kxky_equations <<
>>linearly_coupled_var :: phiMean <<
>>add_BC_for_this_var :: 21 <<
>>new_equation :: 2 <<
>>add_d/dt_term  :: + Iz.Iz.phiMean <<
>>add_rhs_linear :: + D0.phiMean <<
>>add_rhs_NL     :: + D1.oneHalf.phi2 <<
>>====================================<<
```
### End of file
Once the sets of equations have been define, we must indicate that we have reached the end of file by adding the line:
```
>>EOF<<
```

## Closing remarks

In this notebook, we have learned the basics of implementing equations in Coral: 
+ defining parameters
+ distinguishing between linearly terms and quadratic contributions (and how to build the latter)
+ distinguishing between horizontally-averaged components and fluctuations around this mean

However, we worked on a simple example that consisted in a unique equation for a unique variable. In the other notebooks, we ramp up the complexity of the equations by introducing coupled sets of equations (e.g. Navier-Stokes equations).